In [33]:
from knn.classification import KNNClassifier
from knn.nearest_neighbors import NearestNeighborsFinder
import numpy as np

seed = np.random.RandomState(9872)
objectsCount = 10
X = seed.permutation(500).reshape(objectsCount, -1)
labels = [0,0,1,1,1,2,2,3,4,5]
nn = NearestNeighborsFinder(n_neighbors=objectsCount, metric='euclidean')
nn.fit(X)
result = nn.kneighbors(X, return_distance=True)
distances = result[0]
indicies = result[1]
testCount = indicies.shape[0]
labels = np.asarray(labels)
indicies

array([[0, 5, 3, 9, 6, 4, 1, 8, 2, 7],
       [1, 6, 9, 3, 4, 0, 5, 7, 8, 2],
       [2, 4, 9, 8, 7, 1, 0, 3, 5, 6],
       [3, 6, 8, 5, 0, 1, 4, 9, 7, 2],
       [4, 9, 2, 5, 6, 3, 8, 1, 0, 7],
       [5, 6, 0, 3, 4, 8, 1, 9, 2, 7],
       [6, 9, 5, 3, 1, 4, 0, 8, 7, 2],
       [7, 9, 3, 4, 2, 6, 1, 8, 5, 0],
       [8, 3, 9, 4, 6, 5, 0, 2, 1, 7],
       [9, 6, 4, 2, 0, 1, 3, 7, 8, 5]])

In [36]:
labelsTiled = np.tile(labels, (testCount, 1))
labelsTiled

array([[0, 0, 1, 1, 1, 2, 2, 3, 4, 5],
       [0, 0, 1, 1, 1, 2, 2, 3, 4, 5],
       [0, 0, 1, 1, 1, 2, 2, 3, 4, 5],
       [0, 0, 1, 1, 1, 2, 2, 3, 4, 5],
       [0, 0, 1, 1, 1, 2, 2, 3, 4, 5],
       [0, 0, 1, 1, 1, 2, 2, 3, 4, 5],
       [0, 0, 1, 1, 1, 2, 2, 3, 4, 5],
       [0, 0, 1, 1, 1, 2, 2, 3, 4, 5],
       [0, 0, 1, 1, 1, 2, 2, 3, 4, 5],
       [0, 0, 1, 1, 1, 2, 2, 3, 4, 5]])

In [40]:
labelsTiled = np.take_along_axis(labelsTiled, indicies, axis=1)
labelsTiled

array([[0, 2, 1, 5, 2, 1, 0, 4, 1, 3],
       [0, 2, 5, 1, 1, 0, 2, 3, 4, 1],
       [1, 1, 5, 4, 3, 0, 0, 1, 2, 2],
       [1, 2, 4, 2, 0, 0, 1, 5, 3, 1],
       [1, 5, 1, 2, 2, 1, 4, 0, 0, 3],
       [2, 2, 0, 1, 1, 4, 0, 5, 1, 3],
       [2, 5, 2, 1, 0, 1, 0, 4, 3, 1],
       [3, 5, 1, 1, 1, 2, 0, 4, 2, 0],
       [4, 1, 5, 1, 2, 2, 0, 1, 0, 3],
       [5, 2, 1, 1, 0, 0, 1, 3, 4, 2]])

In [41]:
labelsTiled[:,0]

array([0, 0, 1, 1, 1, 2, 2, 3, 4, 5])

In [298]:
!python -m unittest -v tests/test_classification.py

test_base_scenario_cosine_full (tests.test_classification.KNNClassifierTest) ... ok
test_base_scenario_euclidean_full (tests.test_classification.KNNClassifierTest) ... ok
test_base_scenario_identity_cosine (tests.test_classification.KNNClassifierTest) ... ok
test_base_scenario_identity_euclidean (tests.test_classification.KNNClassifierTest) ... ok
test_base_scenario_no_weighted (tests.test_classification.KNNClassifierTest) ... ok
test_base_scenario_weighted (tests.test_classification.KNNClassifierTest) ... ok

----------------------------------------------------------------------
Ran 6 tests in 0.008s

OK


In [43]:
seed = np.random.RandomState(7792)

X = np.zeros(shape=(10, 7))
rows = np.repeat(seed.permutation(10), 3)
cols = seed.choice(X.shape[1], size=len(rows), replace=True)
X[rows, cols] = 1
X = X * (1 + np.arange(X.shape[1]))[np.newaxis]

X_train, X_test = X[:4], X[6:]
y_train = [1, 0, 0, 1]



In [47]:
clf = KNNClassifier(n_neighbors=3, algorithm='my_own', metric='cosine', weights='uniform')
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
y_true = np.asarray([0, 0, 1, 0])

In [69]:
print(X_train)
print(X_test)
y_pred = clf.predict(X_test)
print(y_pred)

[[0. 0. 3. 0. 0. 6. 0.]
 [0. 2. 0. 0. 5. 0. 0.]
 [0. 2. 0. 4. 0. 6. 0.]
 [0. 2. 0. 0. 0. 6. 7.]]
[[0. 2. 0. 0. 5. 6. 0.]
 [0. 2. 0. 4. 0. 0. 0.]
 [0. 0. 0. 0. 5. 6. 7.]
 [0. 2. 0. 0. 5. 6. 0.]]
[[1 0 2]
 [2 1 3]
 [3 0 2]
 [1 0 2]]


In [111]:
distances, indices = clf.kneighbors(X_test, return_distance=True)
testCount = X_test.shape[0]
labelsTiled = np.tile(clf._labels, (testCount, 1))
print(labelsTiled)
print(indices)
print(distances)
labelsMatrix = np.take_along_axis(labelsTiled, indices, axis=1)
print(labelsMatrix)
values = np.apply_along_axis(lambda x: np.argmax(np.bincount(x)), 1, labelsMatrix)
print(values)

[[1 0 0 1]
 [1 0 0 1]
 [1 0 0 1]
 [1 0 0 1]]
[[1 0 2]
 [2 1 3]
 [3 0 2]
 [1 0 2]]
[[0.33205251 0.33435976 0.33700646]
 [0.4023857  0.83390904 0.90519091]
 [0.14093193 0.48831828 0.54131753]
 [0.33205251 0.33435976 0.33700646]]
[[0 1 0]
 [0 0 1]
 [1 1 0]
 [0 1 0]]
[0 0 1 0]


In [ ]:
batch_size = kwargs.pop('batch_size', None)
knn = BatchedKNNClassifier(...)
knn.set_batch_size(batch_size)

In [117]:
gluedLabelsDistancesMatrix = np.hstack((labelsMatrix, distances))
def optLabel(labelsDistances):
    intLabels = self._labels.take(clf._labels[:3].astype(np.int)) 
    distancesSubArray = labelsDistances[3:]
    bc = np.bincount(intLabels, distancesSubArray)
    print(bc)
    return np.argmax(bc)
    
np.apply_along_axis(optLabel, 1, gluedLabelsDistancesMatrix)



[0.         1.         0.         0.33205251 0.33435976 0.33700646]
[0.66905897 0.33435976]
[0.         0.         1.         0.4023857  0.83390904 0.90519091]
[1.23629474 0.90519091]
[1.         1.         0.         0.14093193 0.48831828 0.54131753]
[0.54131753 0.62925021]
[0.         1.         0.         0.33205251 0.33435976 0.33700646]
[0.66905897 0.33435976]


array([0, 0, 1, 0])

In [123]:
np.array([5,4,6,3,1])[np.array([0.,2.]).astype(np.int)]

array([5, 6])

In [ ]:
batch_size = kwargs.pop('batch_size', None)
knn = BatchedKNNClassifier(...)
knn.set_batch_size(batch_size

In [134]:
from sklearn.model_selection import KFold, BaseCrossValidator
from sklearn.metrics import accuracy_score
from sklearn.model_selection import LeaveOneOut, KFold, cross_val_score


In [131]:
cv = KFold(n_splits=5)

In [136]:
from knn.model_selection import knn_cross_val_score

In [137]:
knn_cross_val_score()

TypeError: knn_cross_val_score() missing 4 required positional arguments: 'X', 'y', 'k_list', and 'scoring'

In [141]:
np.sum(np.array([1,2,3]) == np.array([1,2,3]))

3

In [310]:
!python -m unittest -v tests/test_model_selection.py

test_base_scenario (tests.test_model_selection.KnnCrossValScoreTest) ... ok
test_base_scenario_params (tests.test_model_selection.KnnCrossValScoreTest) ... ok
test_leave_one_out_simple_power (tests.test_model_selection.KnnCrossValScoreTest) ... ok
test_leave_one_out_simple_star (tests.test_model_selection.KnnCrossValScoreTest) ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.369s

OK


In [284]:
from sklearn.model_selection import KFold, BaseCrossValidator
from sklearn.metrics import accuracy_score
from sklearn.model_selection import LeaveOneOut, KFold, cross_val_score
from knn.model_selection import knn_cross_val_score
from tests.test_model_selection import knn_cross_val_score_sklearn
from sklearn.neighbors import KNeighborsClassifier

seed = np.random.RandomState(9872)

x = seed.random(size=(200, 10)) * 2 - 1
y = seed.randint(0, 5, size=len(x))

cv = KFold(n_splits=5, shuffle=True, random_state=226)

scores_pred = knn_cross_val_score(x, y, k_list=[3], cv=cv, scoring='accuracy')

scores_true = knn_cross_val_score_sklearn(
    x, y, k_list=[3], cv=cv, scoring='accuracy',
    metric='euclidean', weights='uniform', algorithm='brute',
)

print(scores_pred)
print(scores_true)

defaultdict(<class 'list'>, {3: [0.175, 0.15, 0.175, 0.225, 0.325]})
{3: array([0.225, 0.2  , 0.175, 0.175, 0.375])}


In [157]:
%load_ext autoreload
%autoreload 2

In [280]:
# i = 0
for train_index, test_index in cv.split(x):
    if i > 0:
        break
    X_train, X_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    i += 1

    
    
clf = KNNClassifier(n_neighbors=3, algorithm='brute', metric='euclidean', weights='distance') 
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)


clf2 = KNeighborsClassifier(n_neighbors=3, algorithm='brute', metric='euclidean', weights='distance')
clf2.fit(X_train, y_train)
y_pred2 = clf2.predict(X_test)


print(np.sum(y_pred == y_test))
print(np.sum(y_pred2 == y_test))
print(np.sum(y_pred == y_pred2))
print(accuracy_score)
print(y_pred2)

7
7
40
[2 2 0 2 3 1 1 2 4 3 4 1 0 4 0 1 2 0 2 4 2 3 4 4 2 2 0 0 3 4 2 2 4 3 2 1 4
 3 4 3]
[2 2 0 4 2 1 3 1 2 0 3 2 3 2 2 4 2 4 3 1 2 2 0 2 0 1 1 2 2 1 1 0 1 4 1 3 2
 3 1 2]


In [291]:
print(accuracy_score(y_pred2, y_pred))

1.0


In [282]:
knn_cross_val_score(x, y, k_list=[3], cv=cv, scoring='accuracy')

defaultdict(list, {3: [0.175, 0.15, 0.175, 0.225, 0.325]})

In [293]:
x

array([[-0.93235317, -0.58299408, -0.75874484, ..., -0.8216782 ,
         0.15477339, -0.11654833],
       [ 0.10351144, -0.79517126, -0.20623923, ...,  0.32294897,
         0.24562303, -0.0015621 ],
       [ 0.46014272,  0.79985009,  0.90514771, ..., -0.95247143,
         0.61554454, -0.05252071],
       ...,
       [ 0.44575008, -0.64025086, -0.51592788, ...,  0.97564649,
         0.82068746, -0.52299245],
       [ 0.9147383 , -0.74431902, -0.27130583, ..., -0.04199171,
        -0.88966557,  0.24950883],
       [ 0.16219923, -0.00895498,  0.5134455 , ..., -0.03515586,
         0.79716941, -0.02967496]])

In [296]:
x[:,0]

array([-0.93235317,  0.10351144,  0.46014272, -0.62083452, -0.38969098,
        0.11381843, -0.33002536, -0.75931931, -0.69280168, -0.3004173 ,
       -0.57271476, -0.13000986,  0.97250779,  0.45232553,  0.41675106,
       -0.51235409,  0.90636041,  0.40650687,  0.76460509,  0.65914922,
        0.65500897, -0.20495763,  0.23046336, -0.26701551, -0.32177797,
       -0.05710011,  0.56885869,  0.07465368, -0.4323386 , -0.26306743,
        0.17072263,  0.91213112,  0.6176525 , -0.44939252, -0.60041468,
        0.67886651, -0.70206866, -0.68546758,  0.04948857, -0.39098804,
       -0.78866483, -0.38132403, -0.13204982,  0.99876636, -0.44054774,
        0.6024621 , -0.64703565, -0.71376043,  0.04362155, -0.20778314,
        0.45016975,  0.55265652, -0.91859147,  0.32719594,  0.34760637,
        0.02953929,  0.82914753,  0.03602285,  0.8282147 ,  0.32781723,
       -0.07080675, -0.34190158,  0.5334615 , -0.63804101, -0.56178499,
       -0.36635076,  0.19109025,  0.4896314 , -0.68597522, -0.87